# Force-Field Definition and Reparametrization

In [ ]:
import veloxchem as vlx
import numpy as np

# load B3LYP optimized geometry in the xyz format
molecule = vlx.Molecule.read_xyz_file("data/hs276_optim_b3lyp_dev2-svp.xyz")
molecule.show(atom_indices=True, width=600, height=450)

*This part of the code load the MM Forfield Generator and create an initial topology for the molecule object. By default, RESP charges are computed at the B3LYP/6-31G* as recommended.

In [ ]:
ff_gen = vlx.MMForceFieldGenerator()
ff_gen.create_topology(molecule)
ff_gen.write_gromacs_files('HS-276_initial', 'MOL')

The Force-Field Generator can identify rotatable bonds.

In [ ]:
rot_bonds = ff_gen.rotatable_bonds
print(rot_bonds)

## Reparametrization
We will here focus on the reparametrization of the barrier around the rotatable bond [21, 22]. We use the file 16-21-22-27.xyz from the data folder containing the results from a relaxed scan around rotatable bond [21, 22], i. e. optimized geometries and energies (in the comment line for each geometries).


In [ ]:
ff_gen.reparameterize_dihedrals(rotatable_bond=(21,22), scan_file="data/16-21-22-27.xyz", 
    visualize=True)

The rotational barrier is now much improved but the relative energies between the two conformer minima are still not well reproduced, leading to significant errors in the statistical dihedral distributions. As a remedy to this situation, an additional dihedral potential can be added.

In [ ]:
ff_gen.add_dihedral((16, 21, 22, 27), barrier=0.0, phase=180.0, periodicity=1)

We can now restart the reparametrization procedure with this added dihedral angle.

In [ ]:
ff_gen.reparameterize_dihedrals(rotatable_bond=(21,22), scan_file="data/16-21-22-27.xyz", 
    visualize=True, initial_validation=False)

Another dihedral angle can be added for a better agreement with the QM Potential Energy Surface.

In [ ]:
ff_gen.add_dihedral((16, 21, 22, 27), barrier=0.0, phase=0.0, periodicity=4)

And start a new parametrization with this added dihedral angle.

In [ ]:
ff_gen.reparameterize_dihedrals(rotatable_bond=(21,22), scan_file="data/16-21-22-27.xyz", 
    visualize=True, initial_validation=False)

| Rotatable Bond | Max difference (kJ/mol) | Std deviation (kJ/mol) |
|----------------|-------------------------|------------------------|
| [21, 22]      | 0.477                   | 0.212                  |

With this excellent agreement between the QM and MM Potential Energy Surface, we can now save the topology as, i. e., Gromacs ouptut files.

In [ ]:
ff_gen.write_gromacs_files('HS-276_final', 'MOL')